# 사용자-항목-상호 작용 데이터 검증 및 가져오기 <a class="anchor" id="top"></a>

이 노트북에서는 데이터 세트를 선택하고 Amazon Personalize에 사용할 수 있도록 준비합니다.

1. [소개](#intro)
1. [데이터 세트 또는 데이터 소스 선택](#source)
1. [데이터 준비](#prepare)
1. [데이터 세트 그룹 및 상호 작용 데이터 세트 생성](#group_dataset)
1. [S3 버킷 및 IAM 역할 구성](#bucket_role)
1. [상호 작용 데이터 가져오기](#import)

## 소개 <a class="anchor" id="intro"></a>

대부분의 경우, Amazon Personalize(레시피라고 함)의 알고리즘은 여기서 설명하는 다양한 작업을 처리합니다.

1. **사용자 개인화** - 모든 HRNN 워크플로/사용자 개인화 요구 사항을 지원하는 새로운 릴리스로, 여기서는 이 릴리스를 사용합니다.
1. **HRNN 및 HRNN 메타데이터** - 이전 사용자 상호 작용에 따라 항목을 추천합니다.
1. **HRNN 콜드 스타트** - 아직 상호 작용 데이터가 확보되지 않은 새 항목을 추천합니다.
1. **개인별 순위** - 항목 모음을 선택한 다음 HRNN과 유사한 접근 방식을 사용하여 항목 순서를 지정합니다.
1. **SIMS(유사 항목)** - 특정 항목이 주어지면 사용자가 상호 작용하는 다른 항목도 추천합니다.
1. **인기 카운트** - HRNN 또는 HRNN 메타데이터에서 적절한 정보를 찾을 수 없는 경우 가장 인기 있는 항목을 추천합니다. 이 항목은 기본적으로 반환됩니다.

사용 사례에 관계없이, 모든 알고리즘은 3가지 핵심 속성으로 정의된 사용자-항목-상호 작용 데이터에 대한 학습 기반을 공유합니다.

1. **UserID** - 상호 작용한 사용자
1. **ItemID** - 사용자가 상호 작용한 항목
1. **타임스탬프** - 상호 작용이 발생한 시간

또한 다음과 같이 정의된 이벤트 유형과 이벤트 값도 지원합니다.

1. **이벤트 유형** - 이벤트의 범주형 레이블(찾아보기, 구매, 평점 등)입니다.
1. **이벤트 값** - 발생한 이벤트 유형에 해당하는 값입니다. 간단히 말해, 이벤트 유형에 대해 0과 1 사이의 정규화된 값을 찾는 것입니다. 예를 들어 3단계(클릭, 장바구니에 추가, 구매)로 거래를 완료하는 경우 각 단계마다 event_value는 각각 0.33, 0.66, 1.0이 됩니다.

이벤트 유형 및 이벤트 값 필드는 개인화 모델 훈련을 위해 전송된 데이터를 필터링하는 데 사용할 수 있는 추가 데이터입니다. 이 연습에서는 이벤트 유형이나 이벤트 값이 없습니다.

## 데이터 세트 또는 데이터 소스 선택 <a class="anchor" id="source"></a>
[맨 위로 이동](#top)

앞서 언급했듯이, 사용자-항목-상호 작용 데이터는 서비스를 시작하기 위한 핵심 속성입니다. 즉, 이러한 종류의 데이터를 생성하는 사용 사례를 찾아야 합니다. 몇 가지 일반적인 예는 다음과 같습니다.

1. 비디오 온디맨드 애플리케이션
1. 전자 상거래 플랫폼
1. 소셜 미디어 집계기/플랫폼

Personalize에 적합한 문제의 범위를 지정하는 데 적용되는 몇 가지 지침이 있습니다. [공식 한도](https://docs.aws.amazon.com/personalize/latest/dg/limits.html)는 조금 더 낮지만 아래의 값을 시작점으로 사용하면 유용합니다.

* 인증된 사용자
* 50명 이상의 고유한 사용자
* 100개 이상의 고유한 항목
* 사용자당 24건 이상의 상호 작용 

대부분의 경우 이는 쉽게 달성할 수 있으며 한 범주에서 값이 기준에 못 미치면 다른 범주에서 더 많은 수를 확보하여 보충할 수 있습니다.

일반적으로 데이터는 Personalize에 사용하기에 완벽한 형식으로 제공되지 않으며, 올바르게 구성되기 위해서는 약간의 수정이 필요합니다. 이 노트북에서는 그러한 모든 과정을 안내합니다.

우선, 여기서는 최신 MovieLens 데이터 세트를 사용합니다. 이 데이터 세트에는 2,500만 건 이상의 상호 작용과 항목에 대한 풍부한 메타데이터가 수집되어 있으며, 전체 데이터 세트와 동일한 기능을 적용하면서 훈련 시간을 단축할 수 있는 더 작은 버전의 데이터 세트도 있습니다. 전체 데이터 세트를 사용하려면 USE_FULL_MOVIELENS를 True로 설정합니다.

In [ ]:
USE_FULL_MOVIELENS = False

먼저 아래의 코드를 사용하여 데이터 세트를 다운로드하고 새 폴더에 압축을 풉니다.

In [ ]:
data_dir = "poc_data"
!mkdir $data_dir

if not USE_FULL_MOVIELENS:
    !cd $data_dir && wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
    !cd $data_dir && unzip ml-latest-small.zip
    dataset_dir = data_dir + "/ml-latest-small/"
else:
    !cd $data_dir && wget http://files.grouplens.org/datasets/movielens/ml-25m.zip
    !cd $data_dir && unzip ml-25m.zip
    dataset_dir = data_dir + "/ml-25m/"

다운로드한 데이터 파일을 살펴봅니다.

In [ ]:
!ls $dataset_dir

현재로서는 CSV와 Readme가 몇 개 있다는 것 외에는 알려진 것이 거의 없습니다. 다음으로 Readme을 출력하여 자세히 살펴봅니다.

In [ ]:
!pygmentize $dataset_dir/README.txt

README에서, 상호 작용 데이터의 프록시로 작동해야 하는 `ratings.csv` 파일이 있음을 확인할 수 있습니다. 영화에 평점을 매기는 것도 상호 작용의 한 형태이니까요. 이 데이터 세트에는 일종의 영화 유전자 데이터로서 장르 정보도 일부 포함되어 있습니다. 이 POC에서는 상호 작용 및 장르 데이터에 초점을 맞춥니다.


## 데이터 준비 <a class="anchor" id="prepare"></a>
[맨 위로 이동](#top)

다음으로 데이터를 로드하고 데이터가 양호한 상태인지 확인한 다음, 데이터를 CSV에 저장하여 Amazon Personalize에 사용할 준비가 되도록 합니다.

시작하기 위해 데이터 과학에서 일반적으로 사용되는 Python 라이브러리 컬렉션을 가져옵니다.

In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import boto3
import pandas as pd

그런 다음 데이터 파일을 열고 처음 몇 행을 살펴봅니다.

In [ ]:
original_data = pd.read_csv(dataset_dir + '/ratings.csv')
original_data.head(5)

In [ ]:
original_data.shape

In [ ]:
original_data.describe()

이는 `userId` 및 `movieId` 값의 범위가 충분하다는 것을 보여줍니다. 다음으로, 항상 데이터 형식을 확인하는 것이 좋습니다.

In [ ]:
original_data.info()

In [ ]:
original_data.isnull().any()

이와 관련해, 데이터 세트에 4개의 열에 걸쳐 총 (소규모의 경우 전체 100,836개에 대해 25,000,095개)개의 항목이 있으며 각 셀은 int64 형식으로 저장되지만 평점의 경우만 예외로 float64 형식임을 알 수 있습니다.

int64 형식은 확실히 `userId`와 `movieId`에 적합합니다. 하지만 좀 더 자세히 들어가서 데이터의 타임스탬프를 이해할 필요가 있습니다. Amazon Personalize를 사용하려면 타임스탬프를 [Unix Epoch](https://en.wikipedia.org/wiki/Unix_time) 형식으로 저장해야 합니다.

현재 이 타임스탬프 값은 사람이 읽을 수 없습니다. 그럼 임의의 타임스탬프 값을 가지고 해석하는 방법을 알아보겠습니다.

임의의 타임스탬프를 선택하고 사람이 읽을 수 있는 형식으로 변환하여 변환된 데이터 세트에 대해 간단한 온전성 검사를 수행합니다.

In [ ]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

이 날짜는 타임스탬프로 적절하므로, 나머지 데이터의 형식을 계속 변환할 수 있습니다. 필요한 데이터는 사용자-항목-상호 작용 데이터이며, 이 예에서는 `userId`, `movieId` 및 `timestamp`입니다. 이 데이터 세트에는 `rating`이라는 추가 열이 있으며, 긍정적인 상호 작용을 파악하는 데 활용한 후 데이터 세트에서 삭제할 수 있습니다.

이는 명시적 피드백 영화 평점 데이터 세트이므로 평점 1~5의 영화가 포함되어 있는데, 사용자들이 "좋아하는" 영화만 포함하고 VOD 플랫폼에서 수집되는 데이터에 더 가까운 암시적 데이터 세트를 시뮬레이션하려고 합니다. 이를 위해 5점 만점에 2점 이하인 모든 상호 작용을 걸러내고 "click"이라는 EVENT_Type과 "watch"라는 EVENT_Type을 생성합니다. 그런 다음 평점이 2 이상인 모든 영화에 "click"을, 평점 4 이상인 영화에 "click" 및 "watch"를 할당합니다.

이는 모델링 중인 이벤트와 일치하며, 실제 데이터 세트의 경우 클릭, 시청 횟수 등의 암시적 피드백 및/또는 평점, 좋아요 등의 명시적 피드백을 기반으로 모델링하게 됩니다.

In [ ]:
watched_df = original_data.copy()
watched_df = watched_df[watched_df['rating'] > 3]
watched_df = watched_df[['userId', 'movieId', 'timestamp']]
watched_df['EVENT_TYPE']='watch'
watched_df.head()

In [ ]:
clicked_df = original_data.copy()
clicked_df = clicked_df[clicked_df['rating'] > 1]
clicked_df = clicked_df[['userId', 'movieId', 'timestamp']]
clicked_df['EVENT_TYPE']='click'
clicked_df.head()

In [ ]:
interactions_df = clicked_df.copy()
interactions_df = interactions_df.append(watched_df)
interactions_df.sort_values("timestamp", axis = 0, ascending = True, 
                 inplace = True, na_position ='last') 

In [ ]:
interactions_df.info()

새로운 데이터 세트를 살펴보겠습니다. 

In [ ]:
interactions_df.describe()

데이터를 조작한 후에는 항상 데이터 형식이 변경되었는지 확인합니다.

In [ ]:
interactions_df.dtypes

 Amazon Personalize에는 사용자, 항목 및 타임스탬프의 기본 열 이름이 있습니다. 이러한 기본 열 이름은 `USER_ID`, `ITEM_ID` 및 `TIMESTAMP`입니다. 따라서 데이터 세트의 마지막 수정 사항은 기존 열 헤더를 기본 헤더로 바꾸는 것입니다.

In [ ]:
interactions_df.rename(columns = {'userId':'USER_ID', 'movieId':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True) 

이제 끝났습니다. 이제 데이터가 사용 가능한 상태로 준비되었으므로, CSV 파일로 저장하기만 하면 됩니다.

In [ ]:
interactions_filename = "interactions.csv"
interactions_df.to_csv((data_dir+"/"+interactions_filename), index=False, float_format='%.0f')

## 데이터 세트 그룹 및 상호 작용 데이터 세트 생성 <a class="anchor" id="group_dataset"></a>
[맨 위로 이동](#top)

Amazon Personalize에서 격리 및 추상화의 최상위 수준은 데이터 세트 그룹입니다. 이러한 데이터 세트 그룹 중 하나에 저장된 정보는 다른 데이터 세트 그룹이나 거기에서 생성된 모델에 영향을 주지 않으며 완전히 격리됩니다. 이를 통해 많은 실험을 실행할 수 있으며, 이는 모델을 비공개 상태로 사용자의 데이터에 대해서만 완벽하게 훈련된 상태로 유지하기 위한 방법 중 하나입니다.

이전에 준비된 데이터를 가져오기 전에, 데이터 세트 그룹이 필요하고 해당 데이터 세트 그룹에 상호 작용을 처리하는 데이터 세트를 추가해야 합니다.

데이터 세트 그룹은 다음과 같은 유형의 정보를 저장할 수 있습니다.

* 사용자-항목-상호 작용
* 이벤트 스트림(실시간 상호 작용)
* 사용자 메타데이터
* 항목 메타데이터

상호 작용 데이터의 데이터 세트 그룹과 데이터 세트를 만들기 전에, 사용자 환경이 Amazon Personalize와 정상적으로 통신할 수 있는지 검증해 보겠습니다.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### 데이터 세트 그룹 생성

다음 셀은 `personalize-poc-lastfm`이라는 이름으로 새 데이터 세트 그룹을 만듭니다.

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-poc-movielens"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

데이터 세트 그룹을 사용하려면 먼저 그룹이 활성화되어 있어야 합니다. 활성화하는 데에는 1~2분이 걸립니다. 아래의 셀을 실행하고 활성 상태로 표시될 때까지 기다립니다. 이 셀은 최대 3시간 동안 매초마다 데이터 세트 그룹의 상태를 확인합니다.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

이제 데이터 세트 그룹을 만들었으므로 상호 작용 데이터의 데이터 세트를 만들 수 있습니다.

### 데이터 세트 생성

먼저 업로드하려는 데이터 세트의 유형을 Amazon Personalize에 알려주는 스키마를 정의합니다. 스키마에는 데이터 세트 유형에 따라 몇 가지 예약 키워드와 필수 키워드가 필요합니다. 자세한 내용은 [설명서](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html)를 참조하세요.

여기서는 `USER_ID`, `ITEM_ID` 및 `TIMESTAMP` 필드가 필요한 상호 작용 데이터에 대한 스키마를 만듭니다. 데이터 세트에서 나타나는 순서와 동일한 순서로 스키마에 정의해야 합니다.

In [ ]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-poc-movielens-interactions",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

생성된 스키마를 사용하여 데이터 세트 그룹 내에 데이터 세트를 만들 수 있습니다. 데이터는 아직 로드하지 않는다는 점을 유의하세요. 데이터는 몇 단계 후에 로드합니다.

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-poc-movielens-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## S3 버킷 및 IAM 역할 구성 <a class="anchor" id="bucket_role"></a>
[맨 위로 이동](#top)

지금까지 이 Jupyter 노트북을 실행하는 인스턴스에 연결된 Amazon EBS 인스턴스에 데이터를 다운로드하고 조작하고 저장했습니다. 하지만 Amazon Personalize에는 데이터의 소스 역할을 하는 S3 버킷과 해당 버킷에 액세스하기 위한 IAM 역할이 필요합니다. 이 버킷과 역할을 설정해보겠습니다.

이 Amazon SageMaker 노트북의 기반이 되는 인스턴스에 저장된 메타데이터를 사용하여 해당 노트북이 운영되고 있는 리전을 확인합니다. Amazon SageMaker 외부에서 Jupyter 노트북을 사용하는 경우 아래의 문자열로 리전을 정의하면 됩니다. Amazon S3 버킷은 지금까지 생성한 Amazon Personalize 리소스와 동일한 리전에 있어야 합니다.

In [ ]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

Amazon S3 버킷 이름은 전역적으로 고유합니다. 고유한 버킷 이름을 만들기 위해 아래 코드는 AWS 계정 번호에 `personalizepocvod` 문자열을 추가합니다. 그런 다음 이전 셀에서 확인된 리전에 이 이름으로 버킷을 만듭니다.

In [ ]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalizepocvod"
print(bucket_name)
if region == "us-east-1":
    s3.create_bucket(Bucket=bucket_name)
else:
    s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': region}
        )

### S3에 데이터 업로드

이제 Amazon S3 버킷이 생성되었으므로 사용자-항목-상호 작용 데이터의 CSV 파일을 업로드합니다. 

In [ ]:
interactions_file_path = data_dir + "/" + interactions_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_filename

### S3 버킷 정책 설정
Amazon Personalize가 S3 버킷의 콘텐츠를 읽을 수 있어야 합니다. 이를 허용하는 버킷 정책을 추가합니다.

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

### IAM 역할 생성

Amazon Personalize가 특정 작업을 실행할 수 있는 권한을 가지려면, AWS에서 역할을 수임할 수 있어야 합니다. IAM 역할을 생성하고 필요한 정책을 연결해보겠습니다. 아래의 코드는 과도하게 권한을 부여하는 정책을 연결합니다. 프로덕션 애플리케이션에는 보다 제한적인 정책을 사용하세요.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRolePOC"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

## 상호 작용 데이터 가져오기 <a class="anchor" id="import"></a>
[맨 위로 이동](#top)

앞서 데이터 세트 그룹과 데이터 세트를 만들어 정보를 저장했으므로, 이제 S3 버킷에서 Amazon Personalize 데이터 세트로 데이터를 로드하는 가져오기 작업을 실행합니다. 

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-poc-import1",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

데이터 세트를 사용하려면 먼저 가져오기 작업이 활성 상태여야 합니다. 아래의 셀을 실행하고 활성 상태로 표시될 때까지 기다립니다. 이 셀은 최대 6시간 동안 매초마다 가져오기 작업의 상태를 확인합니다.

데이터 세트의 크기에 따라 데이터를 가져오는 데 시간이 걸릴 수 있습니다. 이 워크숍에서는 데이터 가져오기 작업에 약 15분이 소요됩니다.

In [ ]:
%%time

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

데이터 세트 가져오기가 활성화되면, SIMS, 개인별 순위, 인기 카운트 및 사용자 개인화를 사용하여 모델을 구축할 준비가 된 것입니다. 이 프로세스는 다른 노트북에서도 계속됩니다. 다음 셀을 실행하여 다음 노트북에 사용할 몇 가지 값을 저장하세요.

In [ ]:
%store USE_FULL_MOVIELENS
%store dataset_dir
%store interactions_dataset_arn
%store dataset_group_arn
%store bucket_name
%store role_arn
%store role_name
%store data_dir
%store region
%store interaction_schema_arn